This jupyter notebook visualizes the Positivity Preserving Projection for a simple reaction system A -> A*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

from PositivityPreservingProjection.projections import orthogonal_projection, positivity_projection
from PositivityPreservingProjection.errormetrics import atombalance_consistency

# Atom molecule matrix for system A, A*
E = np.array([[1.0, 1.0]]) # A, A* -> atoms x species

# Initial concentrations of the species
C0 = np.array([0.5, 0.3]) 

# Predicted next concentrations, which do not satisfy the atom balance as compared to C0
C_next_pert = np.array([0.15, 1.1])
dC_pert = C_next_pert - C0

# orthogonal projection 
C_corr_orthogonal = orthogonal_projection(C0, C_next_pert, E)

# check positivity of the corrected concentration
print("Orthogonal projection corrected concentration, with negative concentrations:\n", C_corr_orthogonal)

# projection, with positivity step by step
W = 1/C_next_pert  # Weighting factor
E_w = E @ np.diag(1/W)  # Weighted atom molecule matrix
_, _, Vt = scipy.linalg.svd(E_w)
Rank = np.linalg.matrix_rank(E_w)
B_w = Vt[Rank:].T  # Null space of weighted atom molecule matrix
C0_w = C0 * W  # Weighted initial concentration
C_pert_w = C_next_pert * W  # Weighted perturbed concentration
dC_pert_w = C_pert_w - C0_w  # Weighted change in concentration
dC_corr_w = (dC_pert_w @ B_w) @ B_w.T  # Null space projection in weighted space
C_corr_w = C0* W + dC_corr_w  # Weighted corrected concentration
C_corr_positivity = C_corr_w / W  # Going back to original space

# projection, with positivity using vectorized function
C_corr_positivity_vectorized = positivity_projection(np.expand_dims(C0,0), np.expand_dims(C_next_pert,0), E)

#compare step by step and vectorized function
print("Step by step positivity projection, yielding positive concentrations:\n", C_corr_positivity)
print("Vectorized positivity projection, yielding positive concentrations:\n", C_corr_positivity_vectorized[0])

#check atom balance
print("Orthogonal projection atom balance:\n" , atombalance_consistency(C0, C_corr_orthogonal, E))
print("Positivity projection atom balance:\n" , atombalance_consistency(C0, C_corr_positivity, E))

Then, figure 1 from "Machine Learning Surrogate Models for Mechanistic Kinetics: Embedding Atom Balance and Positivity" is generated 

In [ ]:
# all possibilities for conserving mass balance
dC_A_vals = np.linspace(-2, 2, 10)   # x-axis range
dc_A_star_vals = -dC_A_vals

# all possibilities for conserving mass balance in the concentration space
C_vals_consistent = C0 + np.vstack([dC_A_vals, dc_A_star_vals]).T

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(9, 3))

# --- Left SUBPLOT (State Space) ---
ax1.scatter(C0[0], C0[1], marker='s', label=r'$c_\mathrm{0}$', s=20, c=colors[3], zorder = 5)
ax1.scatter(C_next_pert[0], C_next_pert[1], marker='^', label=r'$\hat{c}_\mathrm{Predicted}$', s=20, c=colors[0], zorder = 5)
ax1.scatter(C_corr_orthogonal[0], C_corr_orthogonal[1], marker='v', label=r'$\tilde{c}_\mathrm{Orthogonal \; Projection}$',s=20, c=colors[1], zorder = 5)
ax1.plot(C_vals_consistent[:, 0], C_vals_consistent[:, 1], color='k', linestyle='-', linewidth=0.5, label=r"Atom conserving $\Delta c$")
ax1.plot([0., 0.0], [-0.5, 1.8], color='magenta', linestyle='--', linewidth=0.1)
ax1.text(-0.075, -0.2, r'$\mathbf{\leftarrow} c_A < 0$', color='k', fontsize=9)
ax1.set_xlim(-0.3, 1.3)
ax1.set_ylim(-0.3, 1.3)
ax1.set_xticks([0, 0.5, 1])
ax1.set_yticks([0, 0.5, 1])

# arrows
ax1.arrow(C_next_pert[0] - 0.025, C_next_pert[1] -0.025, C_corr_orthogonal[0] - C_next_pert[0] + 0.08, C_corr_orthogonal[1] - C_next_pert[1] + 0.08, head_width=0.04, head_length=0.04, linewidth=0.5, color = 'black')
ax1.text(0.05, 1.15, 'Projection', fontsize=9)
ax1.arrow(C0[0] - 0.025, C0[1] + 0.025, C_next_pert[0] - C0[0] + 0.075,  C_next_pert[1] - C0[1] - 0.1, head_width=0.04, head_length=0.04, linewidth=0.5, color = 'black')
ax1.text(0.4, 0.7, 'Model Prediction', fontsize=9)
ax1.set_xlabel(r'$c_A$', fontsize=9)
ax1.set_ylabel(r'$c_{A^*}$', fontsize=9)
ax1.set_title("Orthogonal projection", fontsize=9)

# --- Middle SUBPLOT (scaled space) ---
ax2.scatter(C0_w[0], C0_w[1], marker='s', label=r'Scaled $c_\mathrm{0}$', s=20, c=colors[3], zorder = 5)
ax2.scatter(C_pert_w[0], C_pert_w[1], marker='^', label=r'Scaled $c_\mathrm{Predicted}$', s=20, c=colors[0], zorder = 5)
ax2.scatter(C_corr_w[0], C_corr_w[1], marker='o', label=r'Scaled $\tilde{c}_\mathrm{This \, work}$', s=20, c=colors[2], zorder = 5)
C_vals_consistent_w = C_vals_consistent * W
ax2.plot(C_vals_consistent_w[:, 0], C_vals_consistent_w[:, 1], color='k', linestyle='-', linewidth=0.5, label=r"Atom conserving $\Delta c$")
ax2.plot([0., 0.0], [-1, 8], color='magenta', linestyle='--', linewidth=0.1)
ax2.text(-0.15, 2.9, r'$\mathbf{\leftarrow} c_A < 0$', color='k', fontsize=9)
ax2.set_xlim(-0.25, 3.5)
ax2.set_ylim(-0.25, 3.5)
ax2.set_xticks([0, 1, 2, 3])
ax2.set_yticks([0, 1, 2, 3])

# connecting arrows
ax2.arrow(1.0, 0.925, -0.025, -0.175,head_width=0.1, head_length=0.1, linewidth=0.5, color = 'black', zorder = 10)
ax2.text(0.05, 1.15, 'Projection', fontsize=9)
ax2.arrow(C0_w[0] - 0.025, C0_w[1] + 0.025, C_pert_w[0] - C0_w[0] + 0.25,  C_pert_w[0] - C0_w[1] - 0.1,head_width=0.1, head_length=0.1, linewidth=0.5, color = 'black')
ax2.text(1.5, 0.9, 'Model Prediction', fontsize=9)

ax2.set_xlabel(r'Scaled $c_A$', fontsize=9)
ax2.set_ylabel(r'Scaled $c_{A^*}$', fontsize=9)
ax2.set_title("Orthogonal projection in scaled space", fontsize=9)

# --- Right SUBPLOT (Concentration Space) ---
ax3.scatter(C0[0], C0[1], marker='s', label=r'$c_\mathrm{0}$', s=20, c=colors[3], zorder = 5)
ax3.scatter(C_next_pert[0], C_next_pert[1], marker='^', label=r'$\hat{c}_\mathrm{Predicted}$', s=20, c=colors[0], zorder = 5)
ax3.scatter(C_corr_orthogonal[0], C_corr_orthogonal[1], marker='v', label=r'$\tilde{c}_\mathrm{Orthogonal \; Projection}$',s=20, c=colors[1], zorder = 5)
ax3.scatter(C_corr_positivity[0], C_corr_positivity[1], marker='o', label=r'$\tilde{c}_\mathrm{This \, work}$', s=20, c=colors[2], zorder = 5)
ax3.plot(C_vals_consistent[:, 0], C_vals_consistent[:, 1], color='k', linestyle='-', linewidth=0.5, label=r"Atom conserving $\Delta c$")
ax3.plot([0., 0.0], [-0.5, 1.8], color='magenta', linestyle='--', linewidth=0.1)
ax3.text(-0.05, -0.2, r'$\mathbf{\leftarrow} c_A < 0$', color='k', fontsize=9)
ax3.set_xlabel(r'$c_A$', fontsize=9)
ax3.set_ylabel(r'$c_{A^*}$', fontsize=9)
ax3.set_xlim(-0.3, 1.2)
ax3.set_ylim(-0.3, 1.2)
ax3.set_xticks([0, 0.5, 1])
ax3.set_yticks([0, 0.5, 1])
ax3.set_title("Comparison of projections", fontsize=9)

# connecting arrows
ax3.plot([C_next_pert[0], C_corr_orthogonal[0]], [C_next_pert[1], C_corr_orthogonal[1]], color='black', linestyle='--', linewidth=0.5)
ax3.plot([C_next_pert[0], C_corr_positivity[0]], [C_next_pert[1], C_corr_positivity[1]], color='black', linestyle='--', linewidth=0.5)

# text
ax1.text(-0.5, 1.25, 'a)', fontsize=12)
ax1.text(1.5, 1.25, 'b)', fontsize=12)
ax1.text(3.7, 1.25, 'c)', fontsize=12)
ax1.text(-0.1, 1.6, r'Isomerisation Reaction: $A \rightarrow A^*$, Machine Learning Model Prediction: $\hat{c}_\mathrm{Predicted} \approx f_w(c_0)$', color='k', fontsize=12)
ax3.text(0.3, 0.85, r'Preserve atoms', fontsize=9)
ax3.text(0.4, 0.7, r'\& positivity', fontsize=9)

# legend
ax3.legend(bbox_to_anchor=(-1, -0.2), loc='upper center', ncol=5, frameon=True, edgecolor='black', fontsize=9)

# Adjust spacing between subplots
fig.subplots_adjust(wspace=0.4)
ax1.set_aspect('equal', 'box')
ax2.set_aspect('equal', 'box')
ax3.set_aspect('equal', 'box')